In [2]:
import pandas as pd
import numpy as np


In [3]:
raw_csv_data = pd.read_csv("E:/internship-f1soft/day-7/timeseries-forgit/esewa_airlines.csv")

df = raw_csv_data.copy()
df.head()

,Unnamed: 0,account_id,last_modified_date,travelled_flight_name,total_amount
0,0,1.559470e+19,10/11/2023,BUDDHA AIR,6400.0
1,1,2.693080e+18,10/11/2023,BUDDHA AIR,5200.0
2,2,7.494540e+18,10/11/2023,SHREE AIRLINES,8800.0
3,3,1.493320e+19,10/11/2023,BUDDHA AIR,9001.0
4,4,1.682070e+19,10/11/2023,YETI AIRLINES,23200.0


# Data preprocessing


For rfm we need to check for :
Duplicate records
Missing value

## Duplicate Records checking 

In [5]:
df.duplicated().sum()


0

## check for missing values:

In [7]:
df.isna().sum() # checks for missing values

Unnamed: 0               0
account_id               0
last_modified_date       0
travelled_flight_name    0
total_amount             0
dtype: int64

# Create Recency Frequency Monetery (RFM ) Table

Recency: the recency of purchase/ How many days ago was their last purchase?

Frequency: the frequency of purchases/ total number of purchases/How many times has the customer purchased from our store?

Monetary: the mean monetary value of each purchase/the amount they have spent/How much has this customer spent? Again limit to last two years — or take all time

## Monetery value

In [14]:
# calculating the monetary value of individual customer:
mon_value = df.groupby('account_id')['total_amount'].mean().reset_index()
mon_value

,account_id,total_amount
0,6.630750e+12,4100.000
1,9.564960e+12,4400.000
2,2.564860e+13,3895.000
3,6.785860e+13,7900.000
4,7.441550e+13,7400.000
...,...,...
211504,1.844620e+19,6200.000
211505,1.844640e+19,4400.000
211506,1.844650e+19,4500.000
211507,1.844660e+19,8040.144


## Frequency Calculation

In [18]:
frequency = df.groupby('account_id')['last_modified_date'].count().reset_index()
print(frequency)

          account_id  last_modified_date
0       6.630750e+12                   2
1       9.564960e+12                   1
2       2.564860e+13                   2
3       6.785860e+13                   1
4       7.441550e+13                   1
...              ...                 ...
211504  1.844620e+19                   1
211505  1.844640e+19                   3
211506  1.844650e+19                   1
211507  1.844660e+19                   5
211508  1.844670e+19                   9

[211509 rows x 2 columns]


In [20]:
frequency.rename(columns = {'last_modified_date':'frequency'}, inplace = True)
frequency.head()

,account_id,frequency
0,6.630750e+12,2
1,9.564960e+12,1
2,2.564860e+13,2
3,6.785860e+13,1
4,7.441550e+13,1


## Recency Calculation

For this we need date i.e. last_modified_date in datetime datatype so we first need to check it and if not we need to change to datetime object

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1048575 non-null  int64  
 1   account_id             1048575 non-null  float64
 2   last_modified_date     1048575 non-null  object 
 3   travelled_flight_name  1048575 non-null  object 
 4   total_amount           1048575 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 40.0+ MB


In [22]:
df.last_modified_date= pd.to_datetime(df.last_modified_date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   Unnamed: 0             1048575 non-null  int64         
 1   account_id             1048575 non-null  float64       
 2   last_modified_date     1048575 non-null  datetime64[ns]
 3   travelled_flight_name  1048575 non-null  object        
 4   total_amount           1048575 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 40.0+ MB


In [33]:
recency = df.groupby('account_id')['last_modified_date'].max().reset_index()


recency.head()
# pd.Timestamp.now()

,account_id,last_modified_date
0,6.630750e+12,2024-04-22
1,9.564960e+12,2024-03-21
2,2.564860e+13,2023-02-15
3,6.785860e+13,2024-04-14
4,7.441550e+13,2023-05-08


In [34]:
df['recency']= (pd.Timestamp.now()-recency['last_modified_date']).dt.days
df

,Unnamed: 0,account_id,last_modified_date,travelled_flight_name,total_amount,recency
0,0,1.559470e+19,2023-10-11,BUDDHA AIR,6400.0,16.0
1,1,2.693080e+18,2023-10-11,BUDDHA AIR,5200.0,48.0
2,2,7.494540e+18,2023-10-11,SHREE AIRLINES,8800.0,448.0
3,3,1.493320e+19,2023-10-11,BUDDHA AIR,9001.0,24.0
4,4,1.682070e+19,2023-10-11,YETI AIRLINES,23200.0,366.0
...,...,...,...,...,...,...
1048570,1048570,6.846850e+18,2023-08-07,BUDDHA AIR,5000.0,NaN
1048571,1048571,1.029540e+19,2023-08-07,BUDDHA AIR,5000.0,NaN
1048572,1048572,1.340140e+19,2023-08-07,BUDDHA AIR,4200.0,NaN
1048573,1048573,6.879000e+18,2023-08-07,YETI AIRLINES,9200.0,NaN


In [37]:
del df['recency']
df

,Unnamed: 0,account_id,last_modified_date,travelled_flight_name,total_amount
0,0,1.559470e+19,2023-10-11,BUDDHA AIR,6400.0
1,1,2.693080e+18,2023-10-11,BUDDHA AIR,5200.0
2,2,7.494540e+18,2023-10-11,SHREE AIRLINES,8800.0
3,3,1.493320e+19,2023-10-11,BUDDHA AIR,9001.0
4,4,1.682070e+19,2023-10-11,YETI AIRLINES,23200.0
...,...,...,...,...,...
1048570,1048570,6.846850e+18,2023-08-07,BUDDHA AIR,5000.0
1048571,1048571,1.029540e+19,2023-08-07,BUDDHA AIR,5000.0
1048572,1048572,1.340140e+19,2023-08-07,BUDDHA AIR,4200.0
1048573,1048573,6.879000e+18,2023-08-07,YETI AIRLINES,9200.0


In [49]:
recency.last_modified_date.info()

<class 'pandas.core.series.Series'>
RangeIndex: 211509 entries, 0 to 211508
Series name: last_modified_date
Non-Null Count   Dtype         
--------------   -----         
211509 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 1.6 MB


In [54]:
from datetime import datetime
# recency['recency'] = pd.datetime.now() - recency['last_modified_date']
recency['recency'] = (pd.Timestamp.now() - recency['last_modified_date']).dt.days

In [55]:
recency


,account_id,last_modified_date,recency
0,6.630750e+12,2024-04-22,16
1,9.564960e+12,2024-03-21,48
2,2.564860e+13,2023-02-15,448
3,6.785860e+13,2024-04-14,24
4,7.441550e+13,2023-05-08,366
...,...,...,...
211504,1.844620e+19,2023-10-13,208
211505,1.844640e+19,2024-02-27,71
211506,1.844650e+19,2023-02-27,436
211507,1.844660e+19,2024-04-05,33


In [56]:
# recency.rename(columns = {'last_modified_date':'recency'}, inplace = True)
# recency.head()

since we needed last_modified_date column to calculate recency we can delete it once we have found recency

In [57]:

del recency['last_modified_date']


In [58]:
recency

,account_id,recency
0,6.630750e+12,16
1,9.564960e+12,48
2,2.564860e+13,448
3,6.785860e+13,24
4,7.441550e+13,366
...,...,...
211504,1.844620e+19,208
211505,1.844640e+19,71
211506,1.844650e+19,436
211507,1.844660e+19,33


## Merging the dataframes which we have in seperate dataframes

In [59]:
merged_df = recency.merge(mon_value, on='account_id').merge(frequency, on='account_id')
merged_df


,account_id,recency,total_amount,frequency
0,6.630750e+12,16,4100.000,2
1,9.564960e+12,48,4400.000,1
2,2.564860e+13,448,3895.000,2
3,6.785860e+13,24,7900.000,1
4,7.441550e+13,366,7400.000,1
...,...,...,...,...
211504,1.844620e+19,208,6200.000,1
211505,1.844640e+19,71,4400.000,3
211506,1.844650e+19,436,4500.000,1
211507,1.844660e+19,33,8040.144,5
